In [2]:
import numpy as np
from numpy.linalg import norm
from scipy.linalg import kron
from scipy.stats import unitary_group
from scipy.sparse.linalg import svds, eigsh
import random
from scipy.linalg import expm, sinm, cosm
np.set_printoptions(precision=3)
N = 10 # number of spins
d = 2 # dim

In [3]:
sx = np.array([[0,1],[1,0]])
sy = np.array([[0,-1j],[1j,0]])
sz = np.array([[1,0],[0,-1]])
sxsx = kron(sx, sx)
szsz = kron(sz, sz)
sysy = kron(sy, sy)
h = sxsx + sysy + szsz

In [4]:
U = unitary_group.rvs(2)
UU = kron(U, U)
print((UU.conj().T@szsz@UU))

[[ 0.115+0.j     0.315+0.051j  0.315+0.051j  0.841+0.277j]
 [ 0.315-0.051j -0.115+0.j     0.885+0.j    -0.315-0.051j]
 [ 0.315-0.051j  0.885+0.j    -0.115+0.j    -0.315-0.051j]
 [ 0.841-0.277j -0.315+0.051j -0.315+0.051j  0.115+0.j   ]]


In [28]:
H = 0
for i in range(N-1):
    H += np.kron(np.eye(d**i), np.kron(h, np.eye(d**(N-2-i))))
w, v = eigsh(H, k=1)
print("E/N=", w[0]/N)

E/N= -1.7032140829131521


In [6]:
def random_pauli():
    return random.choice([np.eye(d), sx, sy, sz])
W = random_pauli()
for i in range(N-1):
    W = kron(W, random_pauli())

In [7]:
def make_p(l):
    v = np.array([[1],[0]]) if l[0]==0 else np.array([[0],[1]]) 
    for x in l[1:]:
        qbit = np.array([[1],[0]]) if x==0 else np.array([[0],[1]]) 
        v = kron(v, qbit)
    return v

onep = 0
for i in range(N):
    r = [1 if j==i else 0 for j in range(N)]
    onep = onep + make_p(r)@make_p(r).conj().T

In [8]:
dtheta = 0.01
def random_pauli():
    return random.choice([np.eye(d), sx, sy, sz])
W = random_pauli()
for i in range(N-1):
    W = kron(W, random_pauli())
np.linalg.norm(W - W.conj().T)
F = W@onep@W.conj().T
np.linalg.norm(F - F.conj().T)

0.0

In [29]:
def eval_energy(W):
    psiket = W@make_p([0]*N)
    E = psiket.conj().T@H@psiket
    return E[0][0].real/N

In [40]:
def random_pauli():
    return random.choice([np.eye(d), sx, sy, sz])
W = random_pauli()
for i in range(N-1):
    W = kron(W, random_pauli())


print(eval_energy(W))
F = W@onep@W.conj().T
A = H@F - F@H
Ud = expm(10000000000000000*A)
nW = Ud@W
print(eval_energy(nW))

0.3
0.040600584366774314


In [19]:
zeroket = make_p([0]*N)
psiket = W@zeroket
E = psiket.conj().T@H@psiket
psiket[0]

array([0.+0.j])

In [31]:
Ud

array([[1.-0.j, 0.-0.j, 0.-0.j, ..., 0.-0.j, 0.-0.j, 0.-0.j],
       [0.-0.j, 1.-0.j, 0.-0.j, ..., 0.-0.j, 0.-0.j, 0.-0.j],
       [0.-0.j, 0.-0.j, 1.-0.j, ..., 0.-0.j, 0.-0.j, 0.-0.j],
       ...,
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 1.+0.j]])